<a href="https://colab.research.google.com/github/Kyungmin-Yu/old_datamining/blob/master/catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
os.chdir("/content/drive/My Drive/data")
import pandas as pd
import numpy as np
dat = pd.read_csv("final_data0723_3.csv", encoding='CP949')

In [8]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8758 entries, 0 to 8757
Data columns (total 33 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   y             8686 non-null   float64
 1   date          8758 non-null   object 
 2   month         8758 non-null   int64  
 3   day           8758 non-null   int64  
 4   wday          8758 non-null   object 
 5   hour          8758 non-null   int64  
 6   temp          8758 non-null   float64
 7   rain          8758 non-null   float64
 8   wind_speed    8758 non-null   float64
 9   humidity      8758 non-null   float64
 10  rain_prob     8758 non-null   float64
 11  rain_type     8758 non-null   int64  
 12  skycondition  8758 non-null   int64  
 13  ymd           8758 non-null   object 
 14  daily         8758 non-null   float64
 15  daily_smooth  8758 non-null   float64
 16  pm10_cat      8737 non-null   object 
 17  pm2_5_cat     8737 non-null   object 
 18  altitude      8737 non-null 

In [9]:
dat = dat.set_index(pd.to_datetime(dat['date']))
submit_dat = dat[dat['y'].isnull()]
dat = dat[dat['y'].notnull()]

In [10]:
drop_columns = ['date','day','wday','ymd','daily','rain_prob']
train_1 = dat[(dat["ymd"] < "2020-01-24")].drop(drop_columns,axis=1)
train_2 = dat[(dat["ymd"] < "2020-03-24")].drop(drop_columns,axis=1)
train_3 = dat[(dat["ymd"] < "2020-05-24")].drop(drop_columns,axis=1)

val_1 = dat[((dat["ymd"] > "2020-01-23") & (dat["ymd"] <"2020-01-31"))].drop(drop_columns,axis=1)
val_2 = dat[((dat["ymd"] > "2020-03-23") & (dat["ymd"] <"2020-03-31"))].drop(drop_columns,axis=1)
val_3 = dat[((dat["ymd"] > "2020-05-23") & (dat["ymd"] <"2020-05-31"))].drop(drop_columns,axis=1)

test_1 = submit_dat[(submit_dat["ymd"] == "2020-01-31")].drop(drop_columns,axis=1)
test_2 = submit_dat[(submit_dat["ymd"] == "2020-03-31")].drop(drop_columns,axis=1)
test_3 = submit_dat[(submit_dat["ymd"] == "2020-05-31")].drop(drop_columns,axis=1)

In [11]:
! pip install catboost

     |████████████████████████████████| 64.8MB 61kB/s 


In [12]:
from catboost import CatBoostRegressor

In [13]:
cb_model = CatBoostRegressor(iterations=1000,
                          loss_function='RMSE',
                          eval_metric='RMSE')

In [33]:
from sklearn.metrics import mean_squared_error as mse
def RMSE(y_true,y_pred):
    rmse = (mse(y_true,y_pred))**0.5
    return rmse

In [27]:
def cat_pred (train ,test):
  train_x =  train.drop(['y'],axis=1)
  train_y = train.loc[:,['y']]
  test_x = test.drop(['y'],axis=1)
  test_y = test.loc[:,['y']]
  cat_features = list(train.select_dtypes(include=['object', 'int64']))
  cb_model.fit(train_x, train_y, 
             eval_set=(test_x,test_y), 
             cat_features=cat_features,
             use_best_model=True,
             verbose=False)
  pred_y = cb_model.predict(test_x)
  true_y = test_y.to_numpy().flatten()
  E = lossloss(true_y, pred_y)
 
  return pred_y

In [16]:
def lossloss(y, y_hat):
  result = []
  for i in range(int(y.shape[0]/24)):
    start = 24*i
    end = (24*(i+1))
    tmp = sum(abs(y[start:end] - y_hat[start:end])*y[start:end]/sum(y[start:end]))/133
    result.append(tmp)
  loss = np.mean(result)
  return loss

In [29]:
from sklearn.model_selection import KFold
def cat_fold_prediction(train, test, N_fold=5):
    kf = KFold(n_splits=N_fold, shuffle=True, random_state=0)
    y_pred_mean = np.zeros(len(test))
    for i, (train_index,val_index) in enumerate(kf.split(train)):
        train_cv = train.iloc[train_index,:]
        val_cv = train.iloc[val_index,:]
        y_pred = cat_pred(train_cv, test)
        y_pred_mean += np.array(y_pred)/N_fold
    rmse = RMSE(test['y'],y_pred_mean)
    E = lossloss(test['y'],y_pred_mean)
    print("가중평균: ", E)
    print("RMSE:", rmse) 
    return y_pred_mean




In [ ]:
pred_1 = cat_fold_prediction(train_1, val_1)
pred_2 = cat_fold_prediction(train_2, val_2)
pred_3 = cat_fold_prediction(train_3, val_3)